# Import Libraries

In [21]:
import logging
import tensorflow as tf
from tensorflow.keras.layers import Layer
from tensorflow.keras.models import load_model

# Define Safe Model

In [2]:
safe_model = tf.keras.models.Sequential([
    tf.keras.layers.Input((28, 28, 1)),
    tf.keras.layers.Conv2D(32, (3, 3), activation="relu"),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation="softmax")
])

In [7]:
safe_model_path = "./models/safe_model.h5"
safe_model.save(safe_model_path)

/home/alper/.local/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [11]:
safe_model = load_model(safe_model_path)

# Define Malicious Model

In [5]:
class CustomLayer(Layer):
    def call(self, inputs):
        return inputs * 2 + 1

In [6]:
malicious_model = tf.keras.models.Sequential([
    tf.keras.layers.Input((28, 28, 1)),
    tf.keras.layers.Conv2D(32, (3, 3), activation="relu"),
    CustomLayer(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation="softmax")
])

In [9]:
malicious_model_path = "./models/malicious_model.h5"
malicious_model.save(malicious_model_path)

In [13]:
malicious_model = load_model(malicious_model_path, custom_objects={"CustomLayer": CustomLayer})

# Custom Layer Detection

In [38]:
def detect_custom_layer(model):
    standard_layers = dir(tf.keras.layers)

    custom_layers = []
    for layer in model.layers:
        if type(layer).__name__ not in standard_layers:
            custom_layers.append(type(layer).__name__)
            
    return custom_layers

In [39]:
detect_custom_layer(safe_model)

[]

In [40]:
detect_custom_layer(malicious_model)

['CustomLayer']